In [32]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leonardorangel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/leonardorangel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leonardorangel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
import pandas as pd
import numpy as np
import sys

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix

from scipy import stats as st

from nltk.stem import SnowballStemmer

from statistics import mode

from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt
import re

import json
from sklearn.preprocessing import FunctionTransformer
import joblib

In [34]:
#Cargamos los datos resultantes para comparar con el funcionamiento del pipe
datos_r = pd.read_csv("entrenadoX.csv", sep=',', encoding = 'utf-8')

In [35]:
#Cargamos los datos brutos y los separamos exactamente igual a como se hizo en preparacion
data=pd.read_excel("ODScat_345.xlsx")
data_t, data_v = train_test_split(data, test_size=0.2, random_state=0)

In [36]:
data_t = data_t.drop_duplicates()

In [37]:
data_t.head()

,Textos_espanol,sdg
2651,La movilización de las mujeres en manifestacio...,5
169,El papel combinado como comprador y proveedor ...,3
2993,El desempleo es particularmente alto entre los...,5
1148,"Además, el espíritu empresarial es un vehículo...",4
2270,Aunque Sudáfrica tenía una Oficina de la Condi...,5


In [38]:

def quitarPuntuacion(words):
    new_words = []
    for word in words:
        if word is not None:
            # Adjusted regular expression pattern to exclude colon
            new_word = re.sub(r'[^\w\s:]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

def aMinuscula(words):
    new_words = []
    for word in words:
        if word is not None:
            new_word = word.lower()
            if new_word != ' ':
                new_words.append(new_word)
    return new_words


def eliminarNumeros(words):
    new_words = []
    for word in words:
        if not contieneNumero(word):
            new_words.append(word)
    return new_words

def contieneNumero(s):
    pattern = re.compile(r'\d')
    return bool(pattern.search(s))


spanish_stopwords = set(stopwords.words('spanish'))
def quitarStopwords(words):
    new_words = []
    for word in words:
        if word is not None:
            if word not in spanish_stopwords:
                new_words.append(word)
    return new_words

def preProcesamiento(words):
    words = aMinuscula(words)
    words = eliminarNumeros(words)
    words = quitarPuntuacion(words)
    words = quitarStopwords(words)
    return words

#Esta version elimina convierte a misnusculas, quita puntuacion, quita stopwords
def procesamientoTexto(texto):

    texto['Textos_espanol'] = texto['Textos_espanol'].apply(contractions.fix)
    texto['words'] = texto['Textos_espanol'].apply(word_tokenize)
    texto['words'] = texto['words'].apply(preProcesamiento)
    texto['words'] = texto['words'].apply(lambda x: ' '.join(map(str, x)))
    return texto['words']

In [39]:
procesamientoTexto(data_t)

2651    movilización mujeres manifestaciones callejera...
169     papel combinado comprador proveedor nivel muni...
2993    desempleo particularmente alto jóvenes años im...
1148    además espíritu empresarial vehículo fundament...
2270    aunque sudáfrica oficina condición jurídica so...
                              ...                        
835     compensación enfermedad otorga adultos años mi...
3264    documento cobertura considera fuente secundari...
1653    objetivo análisis identificar puntos común div...
2607    luz aplicación enfoque lente género políticas ...
2732    examinamos resultados ols todas mujeres juntas...
Name: words, Length: 3239, dtype: object

In [40]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

class Transformer_Representacion_Seleccion:
    def __init__(self, count_vectorizer):
        self.count_vectorizer = count_vectorizer
        self.palabras = None
        self.palabras_deseadas = None  # Inicialización de las palabras deseadas

    def fit(self, X, y=None):
        # Ajustar el CountVectorizer y obtener las palabras (características)
        X_transformed = self.count_vectorizer.fit_transform(X)
        self.palabras = self.count_vectorizer.get_feature_names_out()

        # Crear un DataFrame temporal para realizar las selecciones de palabras relevantes
        X_todas_palabras = pd.DataFrame(X_transformed.toarray(), columns=self.palabras)
        
        # Seleccionar las palabras relevantes usando un ciclo tradicional
        self.palabras_deseadas = []
        for nombre in X_todas_palabras.columns:
            # Contar cuántas filas tienen valores distintos de 0 para cada palabra
            dato = (X_todas_palabras[nombre] != 0).sum()
            if dato > 1:
                # Si la palabra aparece en más de una fila, la agregamos a la lista
                self.palabras_deseadas.append(nombre)
        
        return self

    def transform(self, X):
        # Transformar los datos usando CountVectorizer
        X_transformed = self.count_vectorizer.transform(X)
        
        # Crear DataFrame con todas las palabras
        X_todas_palabras = pd.DataFrame(X_transformed.toarray(), columns=self.palabras)
        
        # Seleccionar solo las palabras relevantes
        palabras_a_usar = pd.Index(self.palabras_deseadas).intersection(X_todas_palabras.columns)
        
        # Retornar el DataFrame con las palabras relevantes
        return X_todas_palabras[palabras_a_usar].copy()


In [41]:


# Ajustamos el pipeline con RandomForestClassifier y los mejores parámetros
pipelon = Pipeline([
    ('preprocess', FunctionTransformer(procesamientoTexto)),
    ('representacion', Transformer_Representacion_Seleccion(CountVectorizer())),
    ('clf', RandomForestClassifier(max_depth=42, max_features=102, random_state=111)),
])



In [42]:
pipelon.fit(data_t, data_t['sdg'])

Pipeline(steps=[('preprocess',
                 FunctionTransformer(func=<function procesamientoTexto at 0x16af2c9a0>)),
                ('representacion',
                 <__main__.Transformer_Representacion_Seleccion object at 0x30b900410>),
                ('clf',
                 RandomForestClassifier(max_depth=42, max_features=102,
                                        random_state=111))])

In [45]:
joblib.dump(pipelon, 'pipelon.joblib')

# Later, when testing on new data
# Load the trained pipeline
loaded_pipeline = joblib.load('pipelon.joblib')

In [47]:
#Cargamos los datos de test para probar que funcione bien el pipeline
x_test = pd.read_csv("ValidacionX.csv", sep=',')
y_test = pd.read_csv("ValidacionY.csv", sep=',')
y_test = y_test['sdg']

In [48]:
y_pred = loaded_pipeline.predict(data_v)

# Informe de clasificación
print("Reporte de la Clasificación:\n")
print(classification_report(data_v['sdg'], y_pred))

Reporte de la Clasificación:

              precision    recall  f1-score   support

           3       0.99      0.97      0.98       269
           4       0.97      0.98      0.97       266
           5       0.97      0.98      0.97       275

    accuracy                           0.98       810
   macro avg       0.98      0.98      0.98       810
weighted avg       0.98      0.98      0.98       810

